In [1]:
# 导入常用的库
import os
import cv2
# 导入MindVision品牌的相机库mvsdk
import mvsdk

# 相机持续拍摄并保存照片
def get_capturedImage(imageFilePath=None):
    if not imageFilePath:
        imageFilePath = '../resources/temp.jpg'
    device_list = mvsdk.CameraEnumerateDevice()
    if len(device_list) == 0:
        return None
    device_info = device_list[0]
    cameraIndex = mvsdk.CameraInit(device_info, -1, -1)
    capability = mvsdk.CameraGetCapability(cameraIndex)
    mvsdk.CameraSetTriggerMode(cameraIndex, 0)
    # 加载相机配置文件
    configFilePath = '../resources/camera.Config'
    assert os.path.exists(configFilePath), 'please check if exists %s'%configFilePath
    mvsdk.CameraReadParameterFromFile(cameraIndex, configFilePath)
    # 获取相机拍摄照片的预处理
    mvsdk.CameraPlay(cameraIndex)
    FrameBufferSize = capability.sResolutionRange.iWidthMax * capability.sResolutionRange.iHeightMax * 3
    FrameBuffer_address = mvsdk.CameraAlignMalloc(FrameBufferSize, 16)
    RawData, FrameHead = mvsdk.CameraGetImageBuffer(cameraIndex, 2000)
    mvsdk.CameraImageProcess(cameraIndex, RawData, FrameBuffer_address, FrameHead)
    mvsdk.CameraReleaseImageBuffer(cameraIndex, RawData)
    # 把文件路径转换为绝对文件路径
    imageFilePath_1 = os.path.abspath(imageFilePath)
    status = mvsdk.CameraSaveImage(cameraIndex, imageFilePath_1, FrameBuffer_address, FrameHead, mvsdk.FILE_JPG, 100)
    if status != mvsdk.CAMERA_STATUS_SUCCESS:
        print('ID为%d的相机拍摄并保存照片失败!!!')
        is_successful = False
    else:
        print('ID为%d的相机保存照片至路径：%s' %(cameraIndex, imageFilePath_1))
        is_successful = True
    # 关闭相机、释放帧缓存
    mvsdk.CameraUnInit(cameraIndex)
    mvsdk.CameraAlignFree(FrameBuffer_address)

In [4]:
get_capturedImage('../resources/1.jpg')

ID为3的相机保存照片至路径：C:\Users\Administrator\Desktop\23_git\AI_Projects\006_检测移动物体\resources\1.jpg


In [5]:
get_capturedImage('../resources/2.jpg')

ID为4的相机保存照片至路径：C:\Users\Administrator\Desktop\23_git\AI_Projects\006_检测移动物体\resources\2.jpg
